In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier,RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,\
                            roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix,log_loss, get_scorer_names
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import yaml

In [2]:
df = pd.read_csv("../1.Data/2.Processed/processed2.csv",index_col=0)
df

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Home Ownership,Employment Duration,...,Eficacia pagos respecto financiación inicial,Proporción Morosidad según financiación,Proporción Morosidad según Batch Enrolled,Grado_interes,Grado_interes_ordered,Grade_ordered,SubGrade_ordered,verification_status_ordered,initial_list_status_ordered,application_type_ordered
ID,,,,,,,,,,,,,,,,,,,,,
65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,20.117115,...,0.090884,0.081828,0.095068,Interes medio,0.092253,0.087237,0.093538,0.088663,0.088901,0.092516
1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,4.544139,...,0.064724,0.090536,0.105029,Interes medio,0.092253,0.093896,0.089912,0.095694,0.096714,0.092516
1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,10.438819,...,0.094738,0.081828,0.092179,Interes medio,0.092253,0.103740,0.090244,0.088663,0.088901,0.092516
6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,12.353020,...,0.041446,0.090536,0.092275,Interes alto,0.093069,0.093896,0.094201,0.088663,0.088901,0.092516
14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,5.046181,...,0.011231,0.093986,0.087944,Interes alto,0.093069,0.093896,0.090244,0.088663,0.088901,0.092516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16164945,13601,6848,13175.28583,59,BAT3193689,9.408858,C,A4,MORTGAGE,9.578046,...,0.288985,0.093986,0.087446,Interes bajo,0.092216,0.093896,0.091431,0.088663,0.088901,0.092516
35182714,8323,11046,15637.46301,59,BAT1780517,9.972104,C,B3,RENT,7.471039,...,0.280720,0.093986,0.099459,Interes bajo,0.092216,0.093896,0.082521,0.095694,0.088901,0.092516
16435904,15897,32921,12329.45775,59,BAT1761981,19.650943,A,F3,MORTGAGE,3.971477,...,0.081772,0.153846,0.104027,Interes alto,0.093069,0.091248,0.098616,0.088663,0.088901,0.092516


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67463 entries, 65087372 to 65443173
Data columns (total 48 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Loan Amount                                       67463 non-null  int64  
 1   Funded Amount                                     67463 non-null  int64  
 2   Funded Amount Investor                            67463 non-null  float64
 3   Term                                              67463 non-null  int64  
 4   Batch Enrolled                                    67463 non-null  object 
 5   Interest Rate                                     67463 non-null  float64
 6   Grade                                             67463 non-null  object 
 7   Sub Grade                                         67463 non-null  object 
 8   Home Ownership                                    67463 non-null  object 
 9   Employm

In [4]:
df.loc[:,:"Open Account"].describe()

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Employment Duration,Debit to Income,Delinquency - two years,Inquires - six months,Open Account
count,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000
mean,16848.902776,15770.599114,14621.799323,58.173814,11.846258,9.187942,23.299241,0.327127,0.145754,14.266561
std,8367.865726,8150.992662,6785.345170,3.327441,3.718629,5.136792,8.451824,0.800888,0.473291,6.225060
min,1014.000000,1014.000000,1114.590204,36.000000,5.320006,1.662507,0.675299,0.000000,0.000000,2.000000
25%,10012.000000,9266.500000,9831.684984,58.000000,9.297147,5.896628,16.756416,0.000000,0.000000,10.000000
50%,16073.000000,13042.000000,12793.682170,59.000000,11.377696,7.909632,22.656658,0.000000,0.000000,13.000000
75%,22106.000000,21793.000000,17807.594120,59.000000,14.193533,10.794356,30.048400,0.000000,0.000000,16.000000
max,35000.000000,34999.000000,34999.746430,59.000000,27.182348,46.379368,39.629862,8.000000,5.000000,37.000000


In [5]:
df.loc[:,"Public Record":"Last week Pay"].describe()

,Public Record,Revolving Balance,Revolving Utilities,Total Accounts,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Last week Pay
count,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000
mean,0.081437,7699.342425,52.889443,18.627929,2068.992542,1.143969,59.691578,1.125141,0.021301,71.163260
std,0.346606,7836.148190,22.539450,8.319246,2221.918745,5.244365,357.026346,3.489885,0.144385,43.315845
min,0.000000,0.000000,0.005172,4.000000,4.736746,0.000003,0.000036,0.000036,0.000000,0.000000
25%,0.000000,2557.000000,38.658825,13.000000,570.903814,0.021114,1.629818,0.476259,0.000000,35.000000
50%,0.000000,5516.000000,54.082334,18.000000,1330.842771,0.043398,3.344524,0.780141,0.000000,68.000000
75%,0.000000,10184.500000,69.177117,23.000000,2656.956837,0.071884,5.453727,1.070566,0.000000,105.000000
max,4.000000,116933.000000,100.880050,72.000000,14301.368310,42.618882,4354.467419,166.833000,1.000000,161.000000


In [6]:
df.loc[:,"Last week Pay":"Eficacia pagos respecto financiación inicial"].describe()

,Last week Pay,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status,Funded Amount/Interest Rate,Funded Amount / Debit to Income,Inquires - six months / Open Account,Revolving Balance / Total Revolving Credit Limit,Delinquency - two years / Total Accounts,cuentas públicas prestatario,Eficacia pagos respecto financiación inicial
count,67463.000000,67463.000000,6.746300e+04,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000
mean,71.163260,146.467990,1.595739e+05,23123.005544,0.092510,1474.849009,807.193076,0.011914,0.795545,0.022713,0.102738,0.173749
std,43.315845,744.382233,1.390332e+05,20916.699999,0.289747,947.189719,663.834935,0.041687,1.582962,0.067641,0.376658,0.239998
min,0.000000,1.000000,6.170000e+02,1000.000000,0.000000,65.494750,29.076308,0.000000,0.000000,0.000000,0.000000,0.000164
25%,35.000000,24.000000,5.037900e+04,8155.500000,0.000000,774.270563,388.529612,0.000000,0.121485,0.000000,0.000000,0.038919
50%,68.000000,36.000000,1.183690e+05,16733.000000,0.000000,1213.625096,631.189539,0.000000,0.317982,0.000000,0.000000,0.094562
75%,105.000000,46.000000,2.283750e+05,32146.500000,0.000000,1938.685276,1014.249464,0.000000,0.809357,0.000000,0.000000,0.212829
max,161.000000,16421.000000,1.177412e+06,201169.000000,1.000000,6491.829342,27329.355315,1.000000,58.966197,2.000000,5.000000,8.898065


In [7]:
df.loc[:,"Proporción Morosidad según financiación":].describe()

,Proporción Morosidad según financiación,Proporción Morosidad según Batch Enrolled,Grado_interes_ordered,Grade_ordered,SubGrade_ordered,verification_status_ordered,initial_list_status_ordered,application_type_ordered
count,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000,67463.000000
mean,0.092510,0.092510,0.092510,0.092510,0.092510,0.092510,0.092510,0.092510
std,0.012567,0.007085,0.000393,0.004178,0.006430,0.004476,0.003896,0.000132
min,0.000000,0.078297,0.092216,0.087237,0.078558,0.088663,0.088901,0.089431
25%,0.090536,0.087446,0.092216,0.087237,0.089912,0.088663,0.088901,0.092516
50%,0.093986,0.093750,0.092253,0.093896,0.093538,0.088663,0.088901,0.092516
75%,0.093986,0.096085,0.093069,0.095873,0.095657,0.095694,0.096714,0.092516
max,0.500000,0.125000,0.093069,0.106349,0.130081,0.101551,0.096714,0.092516


In [8]:
X = df.drop([
       'Batch Enrolled', "Grade","Sub Grade","Home Ownership","Verification Status","Loan Title","Initial List Status","Application Type",
       "Grado_interes","Delinquency - two years / Total Accounts","Inquires - six months / Open Account","Loan Status"],axis=1)
y = df["Loan Status"]

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67463 entries, 65087372 to 65443173
Data columns (total 36 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Loan Amount                                       67463 non-null  int64  
 1   Funded Amount                                     67463 non-null  int64  
 2   Funded Amount Investor                            67463 non-null  float64
 3   Term                                              67463 non-null  int64  
 4   Interest Rate                                     67463 non-null  float64
 5   Employment Duration                               67463 non-null  float64
 6   Debit to Income                                   67463 non-null  float64
 7   Delinquency - two years                           67463 non-null  int64  
 8   Inquires - six months                             67463 non-null  int64  
 9   Open Ac

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(53970, 36)
(53970,)
(13493, 36)
(13493,)


In [11]:
under_sampler = RandomUnderSampler(random_state=42)
X_under, y_under = under_sampler.fit_resample(X_train,y_train)

In [12]:
X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(X_under,y_under,test_size=0.2,random_state=42)
print(X_train_under.shape)
print(y_train_under.shape)
print(X_test_under.shape)
print(y_test_under.shape)

(7988, 36)
(7988,)
(1998, 36)
(1998,)


Gradient Boosting

In [13]:
pipegb = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("classifier", GradientBoostingClassifier())
])

gb_params = {
"classifier__n_estimators": [100,150],
'scaler' : [StandardScaler(), None],
'selectkbest__k' : np.arange(5,15),
'classifier__max_features': [3,5,6,7],
'classifier__max_depth': [5,6,7,8]} 

In [14]:
modelgb = GridSearchCV(estimator = pipegb, param_grid = gb_params, cv=5, scoring='neg_log_loss',n_jobs=-1)

In [15]:
get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [16]:
modelgb.fit(X_train_under, y_train_under)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier',
                                        GradientBoostingClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [5, 6, 7, 8],
                         'classifier__max_features': [3, 5, 6, 7],
                         'classifier__n_estimators': [100, 150],
                         'scaler': [StandardScaler(), None],
                         'selectkbest__k': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='neg_log_loss')

In [17]:
print("modelgb best score:",modelgb.best_score_ )
print("modelgb best score:",modelgb.best_params_ )
print("modelgb best score:",modelgb.best_estimator_ )

modelgb best score: -0.7017252876019097
modelgb best score: {'classifier__max_depth': 5, 'classifier__max_features': 3, 'classifier__n_estimators': 100, 'scaler': StandardScaler(), 'selectkbest__k': 14}
modelgb best score: Pipeline(steps=[('scaler', StandardScaler()),
                ('selectkbest', SelectKBest(k=14)),
                ('classifier',
                 GradientBoostingClassifier(max_depth=5, max_features=3))])


In [18]:
y_pred_under_gb = modelgb.best_estimator_.predict(X_test_under)

In [19]:
print("modelgb train recall score", recall_score(y_test_under, y_pred_under_gb))
print("modelgb train precision_score", precision_score(y_test_under, y_pred_under_gb))
print("modelgb train accuracy_score", accuracy_score(y_test_under, y_pred_under_gb))
print("modelgb train f1_score", f1_score(y_test_under, y_pred_under_gb))
print("modelgb train log_loss", log_loss(y_test_under, y_pred_under_gb))


modelgb train recall score 0.4939516129032258
modelgb train precision_score 0.5291576673866091
modelgb train accuracy_score 0.5305305305305306
modelgb train f1_score 0.5109489051094891
modelgb train log_loss 16.921394834330275


In [20]:
print("modelgb train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_gb)

modelgb train confusion matrix


array([[570, 436],
       [502, 490]], dtype=int64)

In [21]:
print("modelgb train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_gb,normalize="true")

modelgb train confusion matrix


array([[0.5666004 , 0.4333996 ],
       [0.50604839, 0.49395161]])

In [22]:
y_pred_gb = modelgb.best_estimator_.predict(X_test)

In [23]:
print("modelgb test recall score", recall_score(y_test, y_pred_gb))
print("modelgb test precision_score", precision_score(y_test, y_pred_gb))
print("modelgb test accuracy_score", accuracy_score(y_test, y_pred_gb))
print("modelgb test f1_score", f1_score(y_test, y_pred_gb))
print("modelgb test log_loss", log_loss(y_test, y_pred_gb))


modelgb test recall score 0.49439102564102566
modelgb test precision_score 0.09848363926576217
modelgb test accuracy_score 0.5346475950492848
modelgb test f1_score 0.16424863569812326
modelgb test log_loss 16.773000787835663


In [24]:
print("modelgb test confusion matrix")
confusion_matrix(y_test, y_pred_gb)

modelgb test confusion matrix


array([[6597, 5648],
       [ 631,  617]], dtype=int64)

In [25]:
print("modelgb test confusion matrix")
confusion_matrix(y_test, y_pred_gb,normalize="true")

modelgb test confusion matrix


array([[0.53875051, 0.46124949],
       [0.50560897, 0.49439103]])

In [26]:
with open("ModeloGB.pkl", 'wb') as archivo_salida:
    pickle.dump(modelgb.best_estimator_, archivo_salida)

In [27]:
with open("ModeloGB.yaml", "w") as f:
    yaml.dump(modelgb.best_params_, f)

Random Forest

In [28]:
piperf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("classifier", RandomForestClassifier())
])
rf_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' : np.arange(5,15),
'classifier__max_features': [3,5,6,7],
'classifier__max_depth': [2,3,4,5,6]
}

In [29]:
modelrf = GridSearchCV(estimator = piperf, param_grid = rf_params, cv=5, scoring='neg_log_loss',n_jobs=-1)

In [30]:
modelrf.fit(X_train_under, y_train_under)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [2, 3, 4, 5, 6],
                         'classifier__max_features': [3, 5, 6, 7],
                         'scaler': [StandardScaler(), None],
                         'selectkbest__k': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='neg_log_loss')

In [31]:
print("modelrf best score:",modelrf.best_score_ )
print("modelrf best score:",modelrf.best_params_ )
print("modelrf best score:",modelrf.best_estimator_ )

modelrf best score: -0.6913330702565003
modelrf best score: {'classifier__max_depth': 4, 'classifier__max_features': 3, 'scaler': StandardScaler(), 'selectkbest__k': 13}
modelrf best score: Pipeline(steps=[('scaler', StandardScaler()),
                ('selectkbest', SelectKBest(k=13)),
                ('classifier',
                 RandomForestClassifier(max_depth=4, max_features=3))])


In [32]:
y_pred_under_rf = modelrf.best_estimator_.predict(X_test_under)

In [33]:
print("modelrf train recall score", recall_score(y_test_under, y_pred_under_rf))
print("modelrf train precision_score", precision_score(y_test_under, y_pred_under_rf))
print("modelrf train accuracy_score", accuracy_score(y_test_under, y_pred_under_rf))
print("modelrf train f1_score", f1_score(y_test_under, y_pred_under_rf))
print("modelrf train log_loss", log_loss(y_test_under, y_pred_under_rf))


modelrf train recall score 0.5272177419354839
modelrf train precision_score 0.5256281407035176
modelrf train accuracy_score 0.529029029029029
modelrf train f1_score 0.5264217413185707
modelrf train log_loss 16.97551443401363


In [34]:
print("modelrf train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_rf)


modelrf train confusion matrix


array([[534, 472],
       [469, 523]], dtype=int64)

In [35]:
print("modelrf train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_rf,normalize="true")

modelrf train confusion matrix


array([[0.53081511, 0.46918489],
       [0.47278226, 0.52721774]])

In [36]:
y_pred_rf = modelrf.best_estimator_.predict(X_test)

In [37]:
print("modelrf test recall score", recall_score(y_test, y_pred_rf))
print("modelrf test precision_score", precision_score(y_test, y_pred_rf))
print("modelrf test accuracy_score", accuracy_score(y_test, y_pred_rf))
print("modelrf test f1_score", f1_score(y_test, y_pred_rf))
print("modelrf test log_loss", log_loss(y_test, y_pred_rf))


modelrf test recall score 0.5376602564102564
modelrf test precision_score 0.1018363939899833
modelrf test accuracy_score 0.5186392944489735
modelrf test f1_score 0.17123899451320657
modelrf test log_loss 17.34999842602208


In [38]:
print("modelrf test confusion matrix")
confusion_matrix(y_test, y_pred_rf)


modelrf test confusion matrix


array([[6327, 5918],
       [ 577,  671]], dtype=int64)

In [39]:
print("modelrf test confusion matrix")
confusion_matrix(y_test, y_pred_rf,normalize="true")

modelrf test confusion matrix


array([[0.51670069, 0.48329931],
       [0.46233974, 0.53766026]])

In [40]:
with open("ModeloRF.pkl", 'wb') as archivo_salida:
    pickle.dump(modelrf.best_estimator_, archivo_salida)
with open("ModeloRF.yaml", "w") as f:
    yaml.dump(modelrf.best_params_, f)

BaggingClassifier

In [41]:
pipebc = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("classifier", BaggingClassifier())
])
bc_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' : np.arange(5,15),
'classifier__max_features': [3,4,5,6],
'classifier__n_estimators': [100,125,150]}

In [42]:
modelBC = GridSearchCV(estimator = pipebc, param_grid = bc_params, cv=5, scoring='neg_log_loss',n_jobs=-1)

In [43]:
modelBC.fit(X_train_under, y_train_under)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 1200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier', BaggingClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__max_features': [3, 4, 5, 6],
                         'classifier__n_estimators': [100, 125, 150],
                         'scaler': [StandardScaler(), None],
                         'selectkbest__k': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='neg_log_loss')

In [44]:
print("modelBC best score:",modelBC.best_score_ )
print("modelBC best score:",modelBC.best_params_ )
print("modelBC best score:",modelBC.best_estimator_ )

modelBC best score: -0.6972608537530963
modelBC best score: {'classifier__max_features': 4, 'classifier__n_estimators': 125, 'scaler': StandardScaler(), 'selectkbest__k': 13}
modelBC best score: Pipeline(steps=[('scaler', StandardScaler()),
                ('selectkbest', SelectKBest(k=13)),
                ('classifier',
                 BaggingClassifier(max_features=4, n_estimators=125))])


In [45]:
y_pred_under_bc = modelBC.best_estimator_.predict(X_test_under)

In [46]:
print("modelBC train recall score", recall_score(y_test_under, y_pred_under_bc))
print("modelBC train precision_score", precision_score(y_test_under, y_pred_under_bc))
print("modelBC train accuracy_score", accuracy_score(y_test_under, y_pred_under_bc))
print("modelBC train f1_score", f1_score(y_test_under, y_pred_under_bc))
print("modelBC train log_loss", log_loss(y_test_under, y_pred_under_bc))

modelBC train recall score 0.5131048387096774
modelBC train precision_score 0.5084915084915085
modelBC train accuracy_score 0.512012012012012
modelBC train f1_score 0.5107877571500251
modelBC train log_loss 17.588869897091698


In [47]:
print("modelBC train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_bc)


modelBC train confusion matrix


array([[514, 492],
       [483, 509]], dtype=int64)

In [48]:
print("modelBC train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_bc,normalize="true")

modelBC train confusion matrix


array([[0.51093439, 0.48906561],
       [0.48689516, 0.51310484]])

In [49]:
y_pred_bc = modelBC.best_estimator_.predict(X_test)

In [50]:
print("modelBC test recall score", recall_score(y_test, y_pred_bc))
print("modelBC test precision_score", precision_score(y_test, y_pred_bc))
print("modelBC test accuracy_score", accuracy_score(y_test, y_pred_bc))
print("modelBC test f1_score", f1_score(y_test, y_pred_bc))
print("modelBC test log_loss", log_loss(y_test, y_pred_bc))


modelBC test recall score 0.47115384615384615
modelBC test precision_score 0.08956587966488956
modelBC test accuracy_score 0.5081153190543245
modelBC test f1_score 0.15051836682452321
modelBC test log_loss 17.729320947422405


In [51]:
print("modelBC test confusion matrix")
confusion_matrix(y_test, y_pred_bc)


modelBC test confusion matrix


array([[6268, 5977],
       [ 660,  588]], dtype=int64)

In [52]:
print("modelBC test confusion matrix")
confusion_matrix(y_test, y_pred_bc,normalize="true")

modelBC test confusion matrix


array([[0.5118824 , 0.4881176 ],
       [0.52884615, 0.47115385]])

In [53]:
with open("modelBC.pkl", 'wb') as archivo_salida:
    pickle.dump(modelBC.best_estimator_, archivo_salida)
with open("modelBC.yaml", "w") as f:
    yaml.dump(modelBC.best_params_, f)

AdaBoost

In [54]:
pipeab = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("classifier", AdaBoostClassifier())
])
ab_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' : np.arange(5,15),
'classifier__learning_rate': [0.1,0.3,0.5,1],
'classifier__n_estimators': [100,125,150]}

In [55]:
modelAB = GridSearchCV(estimator = pipeab, param_grid = ab_params, cv=5, scoring='neg_log_loss',n_jobs=-1)

In [56]:
modelAB.fit(X_train_under, y_train_under)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier', AdaBoostClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__learning_rate': [0.1, 0.3, 0.5, 1],
                         'classifier__n_estimators': [100, 125, 150],
                         'scaler': [StandardScaler(), None],
                         'selectkbest__k': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='neg_log_loss')

In [57]:
print("modelAB best score:",modelAB.best_score_ )
print("modelAB best score:",modelAB.best_params_ )
print("modelAB best score:",modelAB.best_estimator_ )

modelAB best score: -0.692557421781244
modelAB best score: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100, 'scaler': StandardScaler(), 'selectkbest__k': 5}
modelAB best score: Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=5)),
                ('classifier',
                 AdaBoostClassifier(learning_rate=0.1, n_estimators=100))])


In [58]:
y_pred_under_ab = modelAB.best_estimator_.predict(X_test_under)

In [59]:
print("modelAB train recall score", recall_score(y_test_under, y_pred_under_ab))
print("modelAB train precision_score", precision_score(y_test_under, y_pred_under_ab))
print("modelAB train accuracy_score", accuracy_score(y_test_under, y_pred_under_ab))
print("modelAB train f1_score", f1_score(y_test_under, y_pred_under_ab))
print("modelAB train log_loss", log_loss(y_test_under, y_pred_under_ab))

modelAB train recall score 0.5504032258064516
modelAB train precision_score 0.5234899328859061
modelAB train accuracy_score 0.5280280280280281
modelAB train f1_score 0.5366093366093366
modelAB train log_loss 17.011594167135872


In [60]:
print("modelAB train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_ab)


modelAB train confusion matrix


array([[509, 497],
       [446, 546]], dtype=int64)

In [61]:
print("modelAB train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_ab,normalize="true")

modelAB train confusion matrix


array([[0.50596421, 0.49403579],
       [0.44959677, 0.55040323]])

In [62]:
y_pred_ab = modelAB.best_estimator_.predict(X_test)

In [63]:
print("modelAB test recall score", recall_score(y_test, y_pred_ab))
print("modelAB test precision_score", precision_score(y_test, y_pred_ab))
print("modelAB test accuracy_score", accuracy_score(y_test, y_pred_ab))
print("modelAB test f1_score", f1_score(y_test, y_pred_ab))
print("modelAB test log_loss", log_loss(y_test, y_pred_ab))


modelAB test recall score 0.5504807692307693
modelAB test precision_score 0.09927745664739884
modelAB test accuracy_score 0.4964796561179871
modelAB test f1_score 0.16821743388834476
modelAB test log_loss 18.148712749252347


In [82]:
print("modelAB test confusion matrix")
confusion_matrix(y_test, y_pred_ab)


modelAB test confusion matrix


array([[6012, 6233],
       [ 561,  687]], dtype=int64)

In [65]:
print("modelAB test confusion matrix")
confusion_matrix(y_test, y_pred_ab,normalize="true")

modelAB test confusion matrix


array([[0.49097591, 0.50902409],
       [0.44951923, 0.55048077]])

In [66]:
with open("modelAB.pkl", 'wb') as archivo_salida:
    pickle.dump(modelAB.best_estimator_, archivo_salida)
with open("modelAB.yaml", "w") as f:
    yaml.dump(modelAB.best_params_, f)

KNeighborsClassifier

In [67]:
pipeknn = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("classifier", KNeighborsClassifier())
])
knn_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' : np.arange(5,15),
"classifier__n_neighbors": [3,5,7,9],
'classifier__weights': ['uniform', 'distance'],
'classifier__p': [1, 2]}

In [68]:
modelknn = GridSearchCV(estimator = pipeknn, param_grid = knn_params, cv=5, scoring='neg_log_loss',n_jobs=-1)

In [69]:
modelknn.fit(X_train_under, y_train_under)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__n_neighbors': [3, 5, 7, 9],
                         'classifier__p': [1, 2],
                         'classifier__weights': ['uniform', 'distance'],
                         'scaler': [StandardScaler(), None],
                         'selectkbest__k': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
             scoring='neg_log_loss')

In [70]:
print("modelknn best score:",modelknn.best_score_ )
print("modelknn best score:",modelknn.best_params_ )
print("modelknn best score:",modelknn.best_estimator_ )

modelknn best score: -0.7935023655192226
modelknn best score: {'classifier__n_neighbors': 9, 'classifier__p': 2, 'classifier__weights': 'uniform', 'scaler': None, 'selectkbest__k': 13}
modelknn best score: Pipeline(steps=[('scaler', None), ('selectkbest', SelectKBest(k=13)),
                ('classifier', KNeighborsClassifier(n_neighbors=9))])


In [71]:
y_pred_under_knn = modelknn.best_estimator_.predict(X_test_under)

In [72]:
print("modelknn train recall score", recall_score(y_test_under, y_pred_under_knn))
print("modelknn train precision_score", precision_score(y_test_under, y_pred_under_knn))
print("modelknn train accuracy_score", accuracy_score(y_test_under, y_pred_under_knn))
print("modelknn train f1_score", f1_score(y_test_under, y_pred_under_knn))
print("modelknn train log_loss", log_loss(y_test_under, y_pred_under_knn))

modelknn train recall score 0.501008064516129
modelknn train precision_score 0.49354518371400197
modelknn train accuracy_score 0.496996996996997
modelknn train f1_score 0.49724862431215605
modelknn train log_loss 18.130065893925295


In [73]:
print("modelknn train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_knn)


modelknn train confusion matrix


array([[496, 510],
       [495, 497]], dtype=int64)

In [74]:
print("modelknn train confusion matrix")
confusion_matrix(y_test_under, y_pred_under_knn,normalize="true")

modelknn train confusion matrix


array([[0.49304175, 0.50695825],
       [0.49899194, 0.50100806]])

In [75]:
y_pred_knn = modelknn.best_estimator_.predict(X_test)

In [76]:
print("modelknn test recall score", recall_score(y_test, y_pred_knn))
print("modelknn test precision_score", precision_score(y_test, y_pred_knn))
print("modelknn test accuracy_score", accuracy_score(y_test, y_pred_knn))
print("modelknn test f1_score", f1_score(y_test, y_pred_knn))
print("modelknn test log_loss", log_loss(y_test, y_pred_knn))


modelknn test recall score 0.4935897435897436
modelknn test precision_score 0.0904950785955634
modelknn test accuracy_score 0.4943303935373898
modelknn test f1_score 0.15294847920546248
modelknn test log_loss 18.2261800247496


In [77]:
print("modelknn test confusion matrix")
confusion_matrix(y_test, y_pred_knn)


modelknn test confusion matrix


array([[6054, 6191],
       [ 632,  616]], dtype=int64)

In [78]:
print("modelknn test confusion matrix")
confusion_matrix(y_test, y_pred_knn,normalize="true")

modelknn test confusion matrix


array([[0.49440588, 0.50559412],
       [0.50641026, 0.49358974]])

In [79]:
with open("modelknn.pkl", 'wb') as archivo_salida:
    pickle.dump(modelknn.best_estimator_, archivo_salida)
with open("modelknn.yaml", "w") as f:
    yaml.dump(modelknn.best_params_, f)